# Download SOILGRIDS-derived soil texture class map
SOILGRIDS data (Hengl et al., 2017) showing global sand, silt and clay percentages have been downloaded and processed into a global map of USDA soil texture classes. See: https://www.hydroshare.org/resource/1361509511e44adfba814f6950c6e742/

This script downloads and unpacks this global map.


In [1]:
import os
from pathlib import Path
from shutil import copyfile
from datetime import datetime
import pygeohydro as gh
import pygeoutils as geoutils
from pynhd import NLDI
from utils.read_files import read_from_control, make_default_path
import geopandas as gpd
import numpy as np
import xarray as xr
import scipy.stats as sc

#### Control file handling

In [6]:
# Store the name of the 'active' file in a variable
basin = 'TuolumneRiver'
controlFile = f'control_{basin}.txt'

#### Access shapefile geometry

In [7]:
basin_path = read_from_control(controlFile, 'catchment_shp_path')

# Specify the default paths if required 
if basin_path == 'default':
    basin_path = make_default_path('shapefiles/catchment', controlFile) # outputs a Path()
else:
    basin_path = Path(basin_path) # make sure a user-specified path is a Path()

basin_name = f"{basin}.shp"

In [8]:
basin_shp = gpd.read_file(basin_path / basin_name)

#### Find where to save the raw data

In [9]:
# Find the path where the raw soil maps need to go
soil_raw_path = read_from_control(controlFile,'parameter_soil_raw_path')

# Specify the default paths if required 
if soil_raw_path == 'default':
    soil_raw_path = make_default_path('parameters/soilclass/1_soil_classes_global', controlFile) # outputs a Path()
else:
    soil_raw_path = Path(soil_raw_path) # make sure a user-specified path is a Path()

# Make the folder if it doesn't exist
soil_raw_path.mkdir(parents=True, exist_ok=True)

#### Find the path where the subset soil map need to go

In [10]:
# Find the path where the subset soil map need to go
soil_domain_path = read_from_control(controlFile,'parameter_soil_domain_path')
soil_domain_name = read_from_control(controlFile,'parameter_soil_tif_name')

# Specify the default paths if required 
if soil_domain_path == 'default':
    soil_domain_path = make_default_path('parameters/soilclass/2_soil_classes_domain', controlFile) # outputs a Path()
else:
    soil_domain_path = Path(soil_domain_path) # make sure a user-specified path is a Path()

    # Make the folder if it doesn't exist
soil_domain_path.mkdir(parents=True, exist_ok=True)

#### Download the data

In [11]:
def generate_variable_list():
    # prefixes = ["bdod", "cec", "cfvo", "clay", "nitrogen", "ocd", "ocs", "phh2o", "sand", "silt", "soc"]
    prefixes = ["sand", "silt", "clay"]
    depths = [5, 15, 30, 60, 100, 200]
    
    variables = [f"{prefix}_{depth}" for prefix in prefixes for depth in depths]
    return variables

# Example usage
variable_list = generate_variable_list()

In [12]:
if not os.path.exists(soil_raw_path / "raw_soil_properties.nc"):
    soil_ds = gh.soil_soilgrids(layers=variable_list, geometry=basin_shp.geometry[0])
    # save soil_ds to netcdf
    soil_ds.to_netcdf(soil_raw_path / "raw_soil_properties.nc")
else:
    soil_ds = xr.open_dataset(soil_raw_path / "raw_soil_properties.nc")

In [13]:
# Developed from https://www.hydroshare.org/resource/1361509511e44adfba814f6950c6e742/
# Takes sand/silt/clay percentage and returns USDA soil class
def find_usda_soilclass(sand,silt,clay,no_data_value):
    
    # Based on Benham et al., 2009 and matching the following soil class table:
    # SUMMA-ROSETTA-STAS-RUC soil parameter table
    # 1  'CLAY' 
    # 2  'CLAY LOAM'
    # 3  'LOAM' 
    # 4  'LOAMY SAND'
    # 5  'SAND'
    # 6  'SANDY CLAY'
    # 7  'SANDY CLAY LOAM'
    # 8  'SANDY LOAM'
    # 9  'SILT'
    # 10 'SILTY CLAY'
    # 11 'SILTY CLAY LOAM'
    # 12 'SILT LOAM'

    # Initialize a results array
    soilclass = xr.zeros_like(sand)
    
    # Legend
    soiltype = ['clay','clay loam','loam','loamy sand','sand','sandy clay','sandy clay loam','sandy loam','silt','silty clay','silty clay loam','silt loam']
    
    # Apply conditions in reverse order to ensure proper classification
    soilclass = xr.where((sand == no_data_value) | (silt == no_data_value) | (clay == no_data_value), 0, soilclass)
    soilclass = xr.where(((silt >= 50) & (clay >= 12) & (clay < 27)) | ((silt >= 50) & (silt < 80) & (clay < 12)), 12, soilclass)
    soilclass = xr.where((clay >= 27) & (clay < 40) & (sand <= 20), 11, soilclass)
    soilclass = xr.where((clay >= 40) & (silt >= 40), 10, soilclass)
    soilclass = xr.where((silt >= 80) & (clay < 12), 9, soilclass)
    soilclass = xr.where(((clay >= 7) & (clay < 20) & (sand > 52) & ((silt + 2 * clay) >= 30)) | ((clay < 7) & (silt < 50) & ((silt + 2 * clay) >= 30)), 8, soilclass)
    soilclass = xr.where((clay >= 20) & (clay < 35) & (silt < 28) & (sand > 45), 7, soilclass)
    soilclass = xr.where((clay >= 35) & (sand > 45), 6, soilclass)
    soilclass = xr.where(((silt + 1.5 * clay) < 15), 5, soilclass)
    soilclass = xr.where(((silt + 1.5 * clay) >= 15) & ((silt + 2 * clay) < 30), 4, soilclass)
    soilclass = xr.where((clay >= 7) & (clay < 27) & (silt >= 28) & (silt < 50) & (sand < 52), 3, soilclass)
    soilclass = xr.where((clay >= 27) & (clay < 40) & (sand > 20) & (sand <= 45), 2, soilclass)
    soilclass = xr.where((clay >= 40) & (sand <= 45) & (silt < 40), 1, soilclass)
    
    return soilclass, soiltype

In [14]:
# For the specified soil depth ...
# Explicitly create the path
clay_vars = [var for var in list(soil_ds.keys()) if 'clay' in var]
sand_vars = [var for var in list(soil_ds.keys()) if 'sand' in var]
silt_vars = [var for var in list(soil_ds.keys()) if 'silt' in var]

# Get the geotif bands that contain sand/silt/clay and their coordinates
sand = soil_ds[sand_vars]
sand = sand.rename(dict(zip(sand_vars, [v.replace('sand_','') for v in sand_vars])))
silt = soil_ds[silt_vars]
silt = silt.rename(dict(zip(silt_vars, [v.replace('silt_','') for v in silt_vars])))
clay = soil_ds[clay_vars]
clay = clay.rename(dict(zip(clay_vars, [v.replace('clay_','') for v in clay_vars])))

# Specify the 'no data value' 

# Compare sand/silt/clay % to USDA soil triangle and assign soil class
soilclass,_ = find_usda_soilclass(sand,silt,clay,np.nan)

In [15]:
# Extract mode
# Stack variables into a new dimension
stacked = xr.concat([soilclass[var] for var in soilclass.data_vars], dim="var")
stacked.name = "soilclass_mode"

In [16]:
# Compute mode along the "var" dimension
mode_result = sc.mode(stacked, axis=0, keepdims=True)

In [17]:
# Extract mode values
mode_array = xr.DataArray(mode_result.mode.squeeze(), coords=soilclass.coords)
# write the crs and convert to match basin_shp
mode_array = mode_array.rio.write_crs(soil_ds.rio.crs.to_string())
mode_array = mode_array.rio.reproject(basin_shp.crs)
# fillna with 0
mode_array = mode_array.fillna(0)

In [18]:
# Save resulting soil class file as a new .tif, using an existing one as template
soilclass_file = soil_domain_path / soil_domain_name
# save to tif
mode_array.rio.to_raster(soilclass_file)